# Домашнее задание №1: линейная регрессия (10 баллов).

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.


Крупицина - 9 букв


9/4 = 2, остаток 1
#### Напишите сюда свой вариант: 2

In [15]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [16]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

2.0317402001673783e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [18]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

4.9394925357888495e-12


array([-3.60530499e-08, -2.75585696e-08, -3.79586097e-08,  1.52826199e-08,
        3.02967556e-08,  1.68082686e-08,  2.47757284e-08,  9.31137559e+01,
       -1.81654704e-08, -1.02059134e-07,  2.12004461e-08, -4.84129716e-08,
       -1.26132040e-08, -3.69658889e-09,  5.48795955e-08,  1.58661322e-08,
       -4.45097684e-08,  4.00319939e-08,  5.27769951e-08, -3.71487958e-08,
        1.00249014e-07, -4.58265333e-08, -7.79090368e-08, -1.06069161e-08,
        4.54922529e-08, -5.91763205e-09, -6.94299825e-08, -1.43219475e-08,
       -2.27110498e-08,  1.89848607e-08,  1.59668060e-08,  7.59040698e-08,
        3.31719765e+01, -1.32511733e-08,  2.67562575e-08,  1.50675342e-09,
       -2.79731301e-08,  7.89513595e-09,  3.84902806e-09, -1.56587288e-08,
       -4.72320050e-08, -3.49965779e-08,  2.72624481e-08, -1.01060119e-08,
        9.60413480e+01, -3.17252568e-08, -1.51008353e-08,  1.96802629e-08,
        6.94397456e+01, -6.22221973e-09,  3.95981766e-08, -2.74145213e-08,
       -2.89785754e-08, -

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

Разница вызвана тем, что при использовании Градиентного спуска необходимо подбирать гиперпараметры - размер шага, количество итераций, lerning rate. Это связано с тем, что метод градиентного спуска находит в первую очередь локальные минимумы. Поэтому перебор гиперпараметров позволяет найти разные локальные минимумы и выбрать из них самый лучший. Также очень часто градиентный спуск может "не дойти" или наоборот "перешагнуть" минимум,  с этим также позволяет справиться подбор гиперпараметров.

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

In [19]:
for i in [0.00000001, 0.000000001, 0.0000000001, 0.00000000001, 0.000000000001, 0.0000000000001, 0.00000000000001, 0.000000000000001]:
    reg = SGDRegressor(alpha=i).fit(X, y)
    print(mean_squared_error(y, reg.predict(X)))

5.365257477559594e-12
4.895776114152901e-14
4.949083537438866e-16
4.93747841427208e-18
5.671671770690672e-20
1.6541221563899192e-21
9.099347277571536e-25
9.56701103388565e-25


Мы добились примерно того же порядка ответа, поскольку числа максимально близки к нулю, то разница не такая уж и большая.. это все по сути 0

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию. 

In [20]:
class LinearRegression(object):
    def __init__(self, alpha=0.01, l_ratio=0.0001, tol=0.00001, max_iter=7000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
             
    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        
        #добавляем константный признак и инициализируем веса
        
        x_0 = np.ones((X.shape[0], 1))
        X_new = np.column_stack([x_0, X])
        np.random.seed(1234) #фиксим сид чтобы результаты не сбивались от раза к разу
        w_init = np.random.uniform(np.min(X_new), np.max(X_new), (X_new.shape[1]))
        

        
        #для хранения весов
        self.weights = []
        self.weights.append(w_init)
        MSE_last = 0 

        
        for i in range(self.max_iter): #функция остановится на максимуме итераций
            y_pred = (X_new).dot(w_init)
            w_grad = w_init.copy()
            w_grad[-1] = 0
            
            grad = X_new.transpose().dot(X_new.dot(w_init) - y) * 2 / len(y) + 2 * self.l_ratio * w_grad
            w_init = w_init - self.alpha * grad 
            
            MSE = np.sum((y_pred - y) ** 2) / X_new.shape[0] 
            
            if np.abs(MSE - MSE_last) < self.tol:  #еще один критерий остановка если мсе почти перестанет изменяться до конца итераций
                self.weights.append(w_init)
                break
            else:
                self.weights.append(w_init)
                MSE_last = MSE
        
        return self.weights
        
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        x_0 = np.ones((X.shape[0], 1))
        X_new = np.column_stack([x_0, X])
        return np.dot(X_new, self.weights[-1]) 

In [21]:
my_reg = LinearRegression()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

You are amazing! Great work!


In [22]:
mean_squared_error(y, my_reg.predict(X))

0.0009280826600563813

***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [23]:
#l2

from sklearn.linear_model import Ridge

l2_reg = Ridge(alpha = 0.01, tol=0.001, max_iter=1000)
l2_reg.fit(X, y)
print(mean_squared_error(y, l2_reg.predict(X)))

4.9726456105506845e-08


In [24]:
#l1

from sklearn.linear_model import Lasso

l1_reg = Lasso(alpha = 0.01, tol=0.001, max_iter=1000)
l1_reg.fit(X, y)
print(mean_squared_error(y, l1_reg.predict(X)))

0.0009876415896989939


In [25]:
my_reg = LinearRegression(alpha = 0.01)
my_reg.fit(X, y)
print(mean_squared_error(y, my_reg.predict(X)))

0.0009280826600563813


Ну готовая функция оказалась заметно лучше, хотя по сравнению с l1 регуляризацией l2 явно оказывается более успешной. Я предполагаю, что в написанной функции делается большее количество шагов - у нас все таки есть очень определенный критерий, после которого мы останавливаемся. Можно было бы попробовать сделать больше шагов и посмотреть, прийдем ли мы ближе к минимуму.